In [1]:
import os

import utils

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Dataset

- Performance of EPFL Students on the courses 

- 6562 students
- 36 courses
- 7 semesters
- 17490 grades

In [2]:
subset = { 'EE-100', 'EE-102', 'PHYS-101', 'PHYS-106', 'MATH-111', 'MATH-150', 'CS-112', 'CIVIL-123', 'CS-111', 'CIVIL-122', 'ENV-140' }
D = len(subset)
print('NUMBER OF COURSES:', D)

(data, courses) = utils.getSQLData(subset)
print('DATA SHAPE', data.shape)
data.head(10)

NUMBER OF COURSES: 11
DATA SHAPE (12931, 5)


,StudentID,CourseID,ExamDate,Grade,Result
0,4913,9,2013-07-01,3.5,0
1,4983,9,2013-07-01,3.5,0
2,4629,9,2013-07-01,2.5,0
3,4630,9,2013-07-01,1.0,0
4,4631,9,2013-07-01,3.0,0
5,4984,9,2013-07-01,6.0,1
6,4985,9,2013-07-01,4.0,1
7,2201,9,2013-07-01,4.0,1
8,703,9,2013-07-01,1.5,0
9,4632,9,2013-07-01,2.5,0


In [9]:
data[data.StudentID == 1]

,StudentID,CourseID,ExamDate,Grade,Result
3481,1,0,2014-07-01,1.5,0


# Fitting the LFA model

In [4]:
from model import *

model = ModelLFA(D)
model.runData(data[['StudentID', 'CourseID', 'ExamDate', 'Result']].values)

In [11]:
[ (e.date, e.activity.id, e.student.id, e.state ) for e in model.students[4].events ]

[(Timestamp('2014-02-01 00:00:00'), 4, 4, {1: 1, 4: 1}),
 (Timestamp('2014-02-01 00:00:00'), 1, 4, {1: 1, 4: 1}),
 (Timestamp('2014-07-01 00:00:00'), 5, 4, {0: 1, 1: 1, 4: 1, 5: 1}),
 (Timestamp('2014-07-01 00:00:00'), 0, 4, {0: 1, 1: 1, 4: 1, 5: 1}),
 (Timestamp('2015-02-01 00:00:00'), 1, 4, {0: 1, 1: 2, 4: 1, 5: 1}),
 (Timestamp('2015-07-01 00:00:00'), 2, 4, {0: 1, 1: 2, 2: 1, 4: 1, 5: 1})]

In [6]:
l = len(subset)
activities = model.activities.values()

utils.generateGraph('G1', activities, courses, lambda a,b: (1e-2 * a.counts[b]), 'yellow')
utils.generateGraph('G2', activities, courses, lambda a,b: (3e0 * a.params[b]), 'red')

In [7]:
s = { 'CS-111', 'MATH-111', 'CIVIL-123', 'PHYS-101' }

for a in model.activities.values():
    if courses[a.id] in s:
        print('<<<', courses[a.id], '>>>')
        for b in range(a.D):
            if courses[b] in s and b != a.id:
                print('\t', courses[b], '\t', a.params[b], '\t', a.error[b])
        print('\t', 'beta', '\t\t', a.params[-1], '\t', a.error[-1])
        print('-----------------------------------------------')

<<< PHYS-101 >>>
	 CS-111 	 -0.343705531871 	 10.0
	 MATH-111 	 -0.211359199347 	 10.0
	 CIVIL-123 	 0.0 	 10.0
	 beta 		 -0.0735067690757 	 10.0
-----------------------------------------------
<<< CS-111 >>>
	 PHYS-101 	 0.120619373373 	 0.129818293647
	 MATH-111 	 0.227732861482 	 0.142535663827
	 CIVIL-123 	 0.0 	 nan
	 beta 		 0.762709979149 	 0.117446815221
-----------------------------------------------
<<< MATH-111 >>>
	 PHYS-101 	 0.0475796952321 	 10.0
	 CS-111 	 -0.444170111409 	 10.0
	 CIVIL-123 	 0.0 	 10.0
	 beta 		 -0.0406354859829 	 10.0
-----------------------------------------------
<<< CIVIL-123 >>>
	 PHYS-101 	 -0.976005516779 	 10.0
	 CS-111 	 -0.0816732986734 	 10.0
	 MATH-111 	 0.608364609429 	 10.0
	 beta 		 -0.0262830495797 	 10.0
-----------------------------------------------


# Testing Optimizer with simulations

In [16]:
from simulation import * 
from optimizer import *

nA = 11
nAS = 5
nS = 100

def test(OptimizerClass, nA, nAS, nS):
    s = SimulationLFA(nA, model)
    o = OptimizerClass(nA)
    s.runWithOptimizer(o, nAS, nS)
    print(s.evaluateOptimality())

test(RandomOptimizer, nA, nAS, nS)
#test(BanditOptimizer, nA, nAS, nS)

nextActivity 1
nextActivity 3
nextActivity 5
nextActivity 4
nextActivity 8
nextActivity 10
nextActivity 1
nextActivity 8
nextActivity 6
nextActivity 9
nextActivity 9
nextActivity 2
nextActivity 1
nextActivity 0
nextActivity 10
nextActivity 1
nextActivity 6
nextActivity 10
nextActivity 0
nextActivity 5
nextActivity 10
nextActivity 1
nextActivity 1
nextActivity 3
nextActivity 5
nextActivity 3
nextActivity 6
nextActivity 0
nextActivity 7
nextActivity 2
nextActivity 5
nextActivity 6
nextActivity 4
nextActivity 4
nextActivity 6
nextActivity 8
nextActivity 10
nextActivity 1
nextActivity 4
nextActivity 3
nextActivity 6
nextActivity 9
nextActivity 1
nextActivity 7
nextActivity 5
nextActivity 7
nextActivity 1
nextActivity 0
nextActivity 2
nextActivity 4
nextActivity 0
nextActivity 1
nextActivity 3
nextActivity 7
nextActivity 8
nextActivity 4
nextActivity 10
nextActivity 1
nextActivity 0
nextActivity 8
nextActivity 4
nextActivity 1
nextActivity 10
nextActivity 8
nextActivity 8
nextActivity 3
nex